# Array Manipulation and Linear Algebra in BART

BART offers a large suite of numerical array manipulation and linear algebra tools. In this notebook, we will explore some of this array processing.

* Full Tutorials
    * [ ] extract
    * [ ] flatten
    * [ ] flip
    * [x] invert
    * [x] noise
    * [ ] normalize
    * [ ] repmat
    * [ ] reshape
    * [ ] saxpy
    * [x] scale
    * [ ] sdot
    * [ ] std
    * [ ] svd
    * [ ] transpose
    * [x] var
    * [x] vec
    
* **TODO**
    * flags for normalize

In [2]:
bart

BART. Available commands are:
avg         bench       bitmask     cabs        caldir      calmat      
carg        casorati    cc          ccapply     cdf97       circshift   
conj        conv        copy        cpyphs      creal       crop        
delta       ecalib      ecaltwo     estdelay    estdims     estshift    
estvar      extract     fakeksp     fft         fftmod      fftshift    
filter      flatten     flip        fmac        homodyne    invert      
itsense     join        lrmatrix    mandelbrot  mip         nlinv       
noise       normalize   nrmse       nufft       ones        pattern     
phantom     pics        pocsense    poisson     repmat      reshape     
resize      rof         rss         sake        saxpy       scale       
sdot        show        slice       spow        sqpics      squeeze     
std         svd         threshold   toimg       traj        transpose   
twixread    var         vec         version     walsh       wave        
wavelet     wavepsf  

: 1

## Generating Values

### `bart ones`, `bart zeros` and `bart vec`



In [52]:
bart ones -h

Usage: ones dims dim1 ... dimn name

Create an array filled with ones with {dims} dimensions of size {dim1} to {dimn}.

-h		help


`ones` generates an array of all 1's. Similarly, `zeros` gives us an all-zero array of the specified dimensions 

In [66]:
bart ones 1 5 five_ones
bart zeros 1 5 five_zeros

Note that the `show` utility allows us to view the generated values

In [68]:
bart show -h

Usage: show [-m] [-d d] [-s <string>] [-f <string>] <input>

Outputs values or meta data.

-m		show meta data
-d dim      	show size of dimension
-s sep      	use <sep> as the separator
-f format      	use <format> as the format. Default: "%+e%+ei"
-h		help


In [81]:
## TODO Truncate Format String

bart show five_ones

bart show five_zeros

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i


To create a vector of specific values, we can use `bart vec`:

In [130]:
bart vec 1 2 3 4 5 test_vec

bart show test_vec

+1.000000e+00+0.000000e+00i	+2.000000e+00+0.000000e+00i	+3.000000e+00+0.000000e+00i	+4.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i


## Basic Array Manipulation

### `bart normalize`, `bart scale`, `bart invert`

Let's normalize our test vector from above

In [104]:
bart normalize -h

Usage: normalize flags <input> <output>

Normalize along selected dimensions.

-h		help


Note that we use `-b` to specify a bitmask

In [112]:
bart normalize -b 0 test_vec normalized_vec

bart show normalized_vec

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i


Now, we are left with a normalized (unit) vector. Using `bart scale` we can apply a real or complex scalar.

In [107]:
bart scale -h

Usage: scale factor <input> <output>

Scale array by {factor}. The scale factor can be a complex number.

-h		help


In [108]:
bart scale 5 normalized_vec scaled_vec

bart show scaled_vec

+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i


In [111]:
bart scale 1+1i normalized_vec scaled_complex_vec

bart show scaled_complex_vec

+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i


Important: When using a complex number, the imaginary coefficient *must* be included. BART will not recognize `1 + i` as a valid scalar, but `1 + 1i`, as shown above, works perfectly. 

Next, we will use `invert` to perform elementwise inversion of our vector.

In [126]:
bart invert -h 

Usage: invert <input> <output>

Invert array (1 / <input>). The output is set to zero in case of divide by zero.

-h		help


In [129]:
bart invert test_vec inverted_vec

bart show inverted_vec

+1.000000e+00+0.000000e+00i	+5.000000e-01+0.000000e+00i	+3.333333e-01+0.000000e+00i	+2.500000e-01+0.000000e+00i	+2.000000e-01+0.000000e+00i


As described, each element `x` will be replaced by `1 / x`. In the image processing tutorial, we explore the effect of this elementwise transformation on visual data.

## Summary Statistics

### `bart std`, `bart var`

First, we use `bart noise` to apply real-valued noise with a variance of 4 to a vector of all zeros.

In [165]:
bart noise -h

Usage: noise [-s d] [-r] [-n f] <input> <output>

Add noise with selected variance to input.

-s random seed initialization
-r		real-valued input
-n variance      	DEFAULT: 1.0
-h		help


In [173]:
bart ones 1 10 zeros
bart noise -n 4 -r zeros noisy_vec

bart show noisy_vec

+5.672054e-01+0.000000e+00i	+9.672512e-02+0.000000e+00i	+4.566258e+00+0.000000e+00i	-3.144541e+00+0.000000e+00i	-2.936252e+00+0.000000e+00i	+1.935789e+00+0.000000e+00i	-4.396978e+00+0.000000e+00i	+1.351878e+00+0.000000e+00i	+5.980763e-01+0.000000e+00i	+2.039012e+00+0.000000e+00i


Now, we will explore some summary statistics of our noisy data. Note that we use a bitmask to specify the dimensions across which to average or take the variance of. 

In [174]:
bart std $(bart bitmask 0) noisy_vec std

bart show std

+2.770262e+00+0.000000e+00i


In [175]:
bart var $(bart bitmask 0) noisy_vec var

bart show var

+7.674354e+00+0.000000e+00i
